Importing packages

In [15]:
import os
import json
from coupled_modeling import KnowledgeBase

Reading JSON data of an existing coupled system

In [16]:
input_path = os.path.join(
    os.path.abspath(''),
    'mixed_fid_models/',
    'ProjectParametersCoSimFSI.json')

with open(input_path) as f:
    data = json.load(f)

data

{'problem_data': {'start_time': 0.0,
  'end_time': 1.0,
  'echo_level': 0,
  'print_colors': True,
  'parallel_type': 'OpenMP'},
 'solver_settings': {'type': 'coupled_solvers.gauss_seidel_strong',
  'num_coupling_iterations': 20,
  'echo_level': 3,
  'data_transfer_operators': {'direct_transfer': {'type': 'kratos_mapping',
    'mapper_settings': {'mapper_type': 'nearest_neighbor', 'echo_level': 3}}},
  'predictors': [],
  'convergence_accelerators': [{'type': 'aitken',
    'solver': 'low_fid_fluid',
    'data_name': 'pitch_angle'}],
  'convergence_criteria': [{'type': 'relative_norm_previous_residual',
    'solver': 'low_fid_fluid',
    'data_name': 'pitch_angle',
    'abs_tolerance': 1e-07,
    'rel_tolerance': 1e-05}],
  'coupling_sequence': [{'name': 'low_fid_fluid',
    'output_data_list': [],
    'input_data_list': []},
   {'name': 'spring_structure',
    'input_data_list': [{'data': 'lift_force',
      'from_solver': 'low_fid_fluid',
      'from_solver_data': 'lift_force',
      

Connecting to the knowledge base

In [17]:
kb = KnowledgeBase('http://127.0.0.1:5000/api/v1.0/')

Importing data of the existing coupled system

In [18]:
mixed_fid_models = kb.import_coupled_kratos('mixed_fid_models', data)

mixed_fid_models

'instance_42'

Creating a new coupled system

In [19]:
Onera_FSI = kb.create_coupled('Onera_FSI')

Onera_FSI

'instance_63'

Checking properties of the imported coupled system

In [20]:
kb.get_instance_properties(mixed_fid_models)

{'label': ['mixed_fid_models'],
 'problem_data': ['instance_43'],
 'solver_settings': ['instance_44']}

Checking problem data of the imported coupled system

In [21]:
kb.get_instance_properties('instance_2')

{'echo_level': [0],
 'end_time': [1.0],
 'parallel_type': ['OpenMP'],
 'print_colors': [True],
 'start_time': [0.0]}

Copying problem data from the existing coupled system

In [23]:
problem_data = kb.copy_instance(
    'instance_2',
    Onera_FSI,
    {'echo_level': 2})

problem_data

'instance_65'

Checking solver settings of the imported coupled system

In [24]:
kb.get_instance_properties('instance_3')

{'convergence_accelerators': ['instance_6'],
 'convergence_criteria': ['instance_7'],
 'coupling_sequence': ['instance_8', 'instance_9'],
 'data_transfer_operators': ['instance_4'],
 'echo_level': [3],
 'num_coupling_iterations': [20],
 'solvers': ['instance_12', 'instance_17'],
 'type': ['coupled_solvers.gauss_seidel_strong']}

Copying solver settings from the existing coupled system

In [25]:
solver_settings = kb.copy_instance(
    'instance_3',
    Onera_FSI,
    {'echo_level': 4})

solver_settings

'instance_66'

Checking existing convergence accelerators

In [26]:
kb.get_instance_properties('instance_6')

{'data_name': ['pitch_angle'], 'solver': ['low_fid_fluid'], 'type': ['aitken']}

Creating convergence accelerators for the new coupled system from scratch

In [27]:
convergence_accelerators = kb.create_instance(
    'convergence_accelerators',
    solver_settings,
    {'data_name': 'displacements', 'solver': 'CFD', 'type': 'aitken'})

convergence_accelerators

'instance_67'

Checking existing convergence criteria

In [28]:
kb.get_instance_properties('instance_7')

{'abs_tolerance': [1e-07],
 'data_name': ['pitch_angle'],
 'rel_tolerance': [1e-05],
 'solver': ['low_fid_fluid'],
 'type': ['relative_norm_previous_residual']}

Copying convergence criteria

In [29]:
convergence_criteria = kb.copy_instance('instance_7', solver_settings, {
    'data_name': 'displacements',
    'solver': 'CFD',
    'type': 'relative_norm_initial_residual'})

convergence_criteria

'instance_68'

In [ ]:
Checking existing coupling sequences

In [30]:
kb.get_instance_properties('instance_8')

{'name': ['low_fid_fluid']}

In [31]:
kb.get_instance_properties('instance_9')

{'input_data_list': ['instance_10'],
 'name': ['spring_structure'],
 'output_data_list': ['instance_11']}

Creating new coupling sequence

In [32]:
coupling_sequence = kb.create_instance('coupling_sequence', solver_settings, {
    'name': 'CFD'})

coupling_sequence

'instance_69'

Creating another coupling sequence

In [33]:
coupling_sequence = kb.create_instance('coupling_sequence', solver_settings, {
    'name': 'SM'})

coupling_sequence

'instance_70'

Checking existing input data list

In [34]:
kb.get_instance_properties('instance_10')

{'data': ['lift_force'],
 'data_transfer_operator': ['direct_transfer'],
 'from_solver': ['low_fid_fluid'],
 'from_solver_data': ['lift_force']}

Copying input data list

In [ ]:
input_data_list = kb.copy_instance('instance_10', coupling_sequence, {
    'data_transfer_operator': 'mapping_operation',
    'from_solver': 'CFD',
    'data_transfer_operator_options': 'use_transpose'})

Checking existing output data list

In [35]:
kb.get_instance_properties('instance_11')

{'data': ['pitch_angle'],
 'data_transfer_operator': ['direct_transfer'],
 'to_solver': ['low_fid_fluid'],
 'to_solver_data': ['pitch_angle']}

Creating output data list

In [36]:
output_data_list = kb.create_instance('output_data_list', coupling_sequence, {
    'data': 'displacements',
    'data_transfer_operator': 'mapping_operation',
    'to_solver': 'CFD',
    'to_solver_data': 'displacements'})

output_data_list

'instance_71'

Checking existing data transfer operator

In [38]:
kb.get_instance_properties('instance_4')

{'label': ['direct_transfer'],
 'mapper_settings': ['instance_5'],
 'type': ['kratos_mapping']}

Creating mapping operation

In [ ]:
mapping_operation = kb.create_instance('data_transfer_operators', solver_settings, {
    'label': 'mapping_operation',
    'type': 'kratos_mapping'})

mapping_operation

Checking existing mapper settings

In [39]:
kb.get_instance_properties('instance_5')

{'echo_level': [3], 'mapper_type': ['nearest_neighbor']}

Creating mapper settings

In [ ]:
mapper_settings = kb.create_instance('mapper_settings', mapping_operation, {
    'mapper_type': 'nearest_neighbor',
    'use_initial_configuration': True})

mapper_settings

Checking existing solver

In [40]:
kb.get_instance_properties('instance_12')

{'data_': ['instance_15', 'instance_16'],
 'io_settings': ['instance_14'],
 'label': ['low_fid_fluid'],
 'solver_wrapper_settings': ['instance_13'],
 'type': ['solver_wrappers.external.remote_controlled_solver_wrapper']}

Copying solver

In [41]:
CFD = kb.copy_instance('instance_12', solver_settings, {
    'label': 'CFD'})

CFD

'instance_72'

Checking existing data

In [42]:
kb.get_instance_properties('instance_15')

{'label': ['pitch_angle'],
 'location': ['node_historical'],
 'model_part_name': ['single_node_mesh_f'],
 'variable_name': ['SCALAR_DISPLACEMENT']}

In [44]:
kb.get_instance_properties('instance_16')

{'label': ['lift_force'],
 'location': ['node_historical'],
 'model_part_name': ['single_node_mesh_f'],
 'variable_name': ['SCALAR_FORCE']}

Creating data

In [43]:
displacements = kb.create_instance('data', CFD, {
    'label': 'displacements',
    'dimensions': 3,
    'model_part_name': 'WING',
    'variable_name': 'MESH_DISPLACEMENT'})

displacements

'instance_73'

Creating another data

In [45]:
lift_force = kb.create_instance('data', CFD, {
    'label': 'lift_force',
    'dimensions': 3,
    'model_part_name': 'WING',
    'variable_name': 'REACTION'})

lift_force

'instance_74'

Checking existing IO settings

In [46]:
kb.get_instance_properties('instance_14')

{'communication_format': ['file'],
 'connect_to': ['run_fluid'],
 'echo_level': [4],
 'type': ['kratos_co_sim_io']}

Copying io settings

In [ ]:
io_settings = kb.copy_instance('instance_14', CFD, {
    'connect_to': 'run_SU2'})

io_settings

Checking existing solver wrapper settings

In [47]:
kb.get_instance_properties('instance_13')

{'export_data': ['pitch_angle'],
 'import_data': ['lift_force'],
 'import_meshes': ['single_node_mesh_f']}

Copying solver wrapper settings

In [48]:
solver_wrapper_settings = kb.copy_instance('instance_13', CFD, {
    'export_data': 'displacements',
    'import_meshes': 'WING'})

solver_wrapper_settings

'instance_75'

Checking another solver

In [49]:
kb.get_instance_properties('instance_17')

{'data_': ['instance_20', 'instance_21'],
 'io_settings': ['instance_19'],
 'label': ['spring_structure'],
 'solver_wrapper_settings': ['instance_18'],
 'type': ['solver_wrappers.external.remote_controlled_solver_wrapper']}

Creating another solver

In [53]:
SM = kb.create_instance('solvers', solver_settings, {
    'label': 'SM',
    'type': 'solver_wrappers.kratos.structural_mechanics_wrapper'})

SM

'instance_76'

Checking existing data of the second solver

In [50]:
kb.get_instance_properties('instance_20')

{'label': ['pitch_angle'],
 'location': ['node_historical'],
 'model_part_name': ['single_node_mesh_s'],
 'variable_name': ['SCALAR_DISPLACEMENT']}

In [51]:
kb.get_instance_properties('instance_21')

{'label': ['lift_force'],
 'location': ['node_historical'],
 'model_part_name': ['single_node_mesh_s'],
 'variable_name': ['SCALAR_FORCE']}

Coping data for the second solver from the first one

In [54]:
displacements = kb.copy_instance(displacements, SM, {
    'model_part_name': 'Structure.interface',
    'variable_name': 'DISPLACEMENT'})

displacements

'instance_77'

Copying another data from the first solver

In [55]:
lift_force = kb.copy_instance(lift_force, SM, {
    'model_part_name': 'Structure.interface',
    'variable_name': 'POINT_LOAD'})

lift_force

'instance_78'

Checking existing wrapper settings of the second solver

In [56]:
kb.get_instance_properties('instance_18')

{'export_data': ['lift_force'],
 'import_data': ['pitch_angle'],
 'import_meshes': ['single_node_mesh_s']}

Creating solver wrapper settings for the second solver

In [57]:
solver_wrapper_settings = kb.create_instance('solver_wrapper_settings', SM, {
    'input_file': 'ProjectParametersSM'})

solver_wrapper_settings

'instance_79'

Exporting created coupled system in JSON

In [59]:
export = kb.export_coupled_kratos(Onera_FSI)

export

{'problem_data': {'echo_level': 2,
  'end_time': 1.0,
  'parallel_type': 'OpenMP',
  'print_colors': True,
  'start_time': 0.0},
 'solver_settings': {'convergence_accelerators': [{'data_name': 'displacements',
    'solver': 'CFD',
    'type': 'aitken'}],
  'convergence_criteria': [{'abs_tolerance': 1e-07,
    'data_name': 'displacements',
    'rel_tolerance': 1e-05,
    'solver': 'CFD',
    'type': 'relative_norm_initial_residual'}],
  'coupling_sequence': [{'name': 'CFD'},
   {'name': 'SM',
    'output_data_list': [{'data': 'displacements',
      'data_transfer_operator': 'mapping_operation',
      'to_solver': 'CFD',
      'to_solver_data': 'displacements'}]}],
  'echo_level': 4,
  'num_coupling_iterations': 20,
  'solvers': {'CFD': {'data': {'displacements': {'dimensions': 3,
      'model_part_name': 'WING',
      'variable_name': 'MESH_DISPLACEMENT'},
     'lift_force': {'dimensions': 3,
      'model_part_name': 'WING',
      'variable_name': 'REACTION'}},
    'solver_wrapper_setti

Saving exported data into a file

In [60]:
with open('export_onera_fsi.json', 'w') as file:
    json.dump(export, file, indent=2)

Inferring new classes from the created coupled system

In [64]:
kb.infer_coupled_structure(Onera_FSI)

400

Updating the knowledge base

In [65]:
kb.save()

201